In [2]:
import plotly
plotly.__version__

'5.10.0'

In [10]:
# sure in-use
from dash import Dash, html, dcc
import plotly.express as px
import pandas as pd
import plotly.graph_objs as go
from dash import Dash, dcc, html, Input, Output

# not sure in-use
import chart_studio
import chart_studio.dashboard_objs as dashboard
import chart_studio.plotly as py


from chart_studio.plotly import plot, iplot

from datetime import datetime
import IPython.display
from IPython.display import Image
import numpy as np
import pandas_datareader.data as web

import urllib3
urllib3.disable_warnings()

from plotly_calplot import calplot
import re


In [4]:
# connect this notebook to plotly aacount

username = 'zhongming.shi' 
api_key = 'jRq7UaGcQa50yr2R8PvO'

chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

In [5]:
colors = ['#3f545b', '#cd6116', '#f1bc1a', '#f6ebb6', '#a3d6ca', '#3290b5', '#03608c'],  #BIA colors
colorscale = 'Reds'

In [6]:
# create the plots

cal_start_date = "2022-01-01"
cal_end_date = "2022-12-31"

path = r'/Users/shi_zhongming/Dropbox/SG data/data_gov/place/sorted/usecase/cea/simulations/BIA_clementi/outputs/data/potentials/agriculture/plot/B1000_BIA_visual_AmaranthRed.csv'
df = pd.read_csv(path).iloc[3:,[38]]

df_to_visual = pd.DataFrame({
    "ds": pd.date_range(cal_start_date, cal_end_date),
    "value": df['37'].values.astype(float)
})


# plot the calendar
fig_calendar = calplot(
    df_to_visual,
    x="ds",
    y="value",
    years_title=True,
    colorscale=colorscale,
    dark_theme=False,
    gap=1,
    name="Data",
    month_lines_width=2, 
    month_lines_color="#fff",
)

py.iplot(fig_calendar, filename = 'calendar', auto_open=False)

In [7]:
app = Dash(__name__)

# -- Import and clean data (importing csv into pandas)
# df = pd.read_csv("intro_bees.csv")
df = pd.read_csv("https://raw.githubusercontent.com/Coding-with-Adam/Dash-by-Plotly/master/Other/Dash_Introduction/intro_bees.csv")

df = df.groupby(['State', 'ANSI', 'Affected by', 'Year', 'state_code'])[['Pct of Colonies Impacted']].mean()
df.reset_index(inplace=True)
print(df[:5])

# ------------------------------------------------------------------------------
# App layout
app.layout = html.Div([

    html.H1("Web Application Dashboards with Dash", style={'text-align': 'center'}),

    dcc.Dropdown(id="slct_year",
                 options=[
                     {"label": "2015", "value": 2015},
                     {"label": "2016", "value": 2016},
                     {"label": "2017", "value": 2017},
                     {"label": "2018", "value": 2018}],
                 multi=False,
                 value=2015,
                 style={'width': "40%"}
                 ),

    html.Div(id='output_container', children=[]),
    html.Br(),

    dcc.Graph(id='my_bee_map', figure={})

])


     State  ANSI Affected by  Year state_code  Pct of Colonies Impacted
0  Alabama     1     Disease  2015         AL                      0.05
1  Alabama     1     Disease  2016         AL                      1.20
2  Alabama     1     Disease  2017         AL                      2.25
3  Alabama     1     Disease  2018         AL                      1.30
4  Alabama     1     Disease  2019         AL                      1.80


In [8]:
# App layout

app.layout = html.Div([

    html.H1("Web Application Dashboards with Dash", style={'text-align': 'center'}),

    dcc.Dropdown(id="slct_year",
                 options=[
                     {"label": "2015", "value": 2015},
                     {"label": "2016", "value": 2016},
                     {"label": "2017", "value": 2017},
                     {"label": "2018", "value": 2018}],
                 multi=False,
                 value=2015,
                 style={'width': "40%"}
                 ),

    html.Div(id='output_container', children=[]),
    html.Br(),

    dcc.Graph(id='my_bee_map', figure={})

])


In [15]:
# Connect the Plotly graphs with Dash Components
@app.callback(
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='my_bee_map', component_property='figure')],
    [Input(component_id='slct_year', component_property='value')]
)
def update_graph(option_slctd):
    print(option_slctd)
    print(type(option_slctd))

    container = "The year chosen by user was: {}".format(option_slctd)

    dff = df.copy()
    dff = dff[dff["Year"] == option_slctd]
    dff = dff[dff["Affected by"] == "Varroa_mites"]

    # Plotly Express
    fig = px.choropleth(
        data_frame=dff,
        locationmode='USA-states',
        locations='state_code',
        scope="usa",
        color='Pct of Colonies Impacted',
        hover_data=['State', 'Pct of Colonies Impacted'],
        color_continuous_scale=px.colors.sequential.YlOrRd,
        labels={'Pct of Colonies Impacted': '% of Bee Colonies'},
        template='plotly_dark'
    )

    # Plotly Graph Objects (GO)
    # fig = go.Figure(
    #     data=[go.Choropleth(
    #         locationmode='USA-states',
    #         locations=dff['state_code'],
    #         z=dff["Pct of Colonies Impacted"].astype(float),
    #         colorscale='Reds',
    #     )]
    # )
    #
    # fig.update_layout(
    #     title_text="Bees Affected by Mites in the USA",
    #     title_xanchor="center",
    #     title_font=dict(size=24),
    #     title_x=0.5,
    #     geo=dict(scope='usa'),
    # )

    return container, fig

if __name__ == '__main__':
    app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: on


Address already in use
Port 8050 is in use by another program. Either identify and stop that program, or start the server with a different port.


AssertionError: 